In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


from keras.layers import Input, Dense, Flatten,Activation
from keras.layers import Conv1D, MaxPooling1D,Dropout,LSTM
from keras.layers import Embedding

from numpy import zeros
from numpy import asarray

from keras.preprocessing.text import Tokenizer

from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential

from keras.layers import Dense
from keras.layers import Flatten

from keras.layers.embeddings import Embedding

from gensim.models import FastText

In [2]:
sentences = open('../data/dataset_for_char_rnn.txt').readlines()

In [3]:
for i in range(len(sentences)):
    sentences[i] = sentences[i].strip().split(' ')
sentences[:3]

[['амаравкэваратэн', 'таа’койӈын'],
 ['йъйыӄык', 'ныӄэԓпэратӄэн', 'вытэчгытрыӄэргыԓьын', 'йыӈэттэт'],
 ['мыкыӈ', 'нывытрэтӄин', 'чеԓгатвытрыԓьо', 'ынӄорыым', 'вытэчгытрыԓьо']]

In [4]:
fasttext_model = FastText(size=10, window=4, min_count=1)
fasttext_model.build_vocab(sentences)
fasttext_model.train(sentences, total_examples=len(sentences), epochs=10)


In [5]:
fasttext_model['йъйыӄык']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.76831186,  0.2053792 ,  0.6830166 ,  0.62832826, -2.1753614 ,
       -0.83871263, -1.3315121 ,  0.34191957,  1.094012  ,  0.79018897],
      dtype=float32)

In [6]:
result_dict_common = dict()

training_file = '../data/train.tsv'

for i, line in enumerate(open(training_file).readlines()):
    row = line.strip().split('\t')
    tokens = row[0]
    for id_tok, tok in enumerate(tokens.split(' ')):
        if id_tok in result_dict_common:
            result_dict_common[id_tok].append(tok)
        else:
            result_dict_common[id_tok] = list()
            result_dict_common[id_tok].append(tok)

In [7]:
df = pd.read_csv('../data/df_for_train_classifire.csv', index_col=0)

In [8]:
df.head(2)

,type,text,label
0,2,амаравкэваратэн,таа’койӈын
1,2,йъйыӄык,ныӄэԓпэратӄэн


In [9]:
df.type.value_counts()[:5]

2    26473
3    20768
4    15301
5    10986
6     7741
Name: type, dtype: int64

In [10]:
result_dict = dict()

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.text.tolist())

vocab_size = len(tokenizer.word_index) + 1
word_index = tokenizer.word_index
EMBEDDING_DIM = 10

embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))

for word, i in word_index.items():
    try:
        embedding_vector = fasttext_model[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)


all_types = pd.unique(df.type).tolist()

for id_t, t in enumerate(all_types):
    print('Current type', id_t, 'All type', len(all_types))
    tmp_df = df[df.type == t]
    tmp_df = tmp_df.drop(['type'], axis=1)

    result_dict[t] = dict()
    result_dict[t]['le'] = LabelEncoder()
    tmp_df['label'] = result_dict[t]['le'].fit_transform(tmp_df['label'])

    labels_len = len(pd.unique(tmp_df['label']).tolist())

    encoded_docs = tokenizer.texts_to_sequences(tmp_df.text.tolist())
    
    padded_docs = pad_sequences(encoded_docs, maxlen=t-1, padding='post')

    labels = to_categorical(np.asarray(tmp_df['label'].tolist()))


    # result_dict[t]['model'] = Sequential()
    # result_dict[t]['model'].add(Embedding(result_dict[t]['vocab_size'], 10, input_length=t-1))
    # result_dict[t]['model'].add(Activation('relu'))
    # result_dict[t]['model'].add(Dropout(0.1))
    # result_dict[t]['model'].add(Flatten())
    # result_dict[t]['model'].add(Dense(30))
    # result_dict[t]['model'].add(Activation('relu'))
    # result_dict[t]['model'].add(Dense(labels_len))
    # result_dict[t]['model'].add(Activation('softmax'))
    # result_dict[t]['model'].compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # result_dict[t]['model'].fit(padded_docs, labels, epochs=5, verbose=1)

    if id_t > 15:
        epoh_n = 10
    else:
        epoh_n = 50

    result_dict[t]['model'] = Sequential()
    result_dict[t]['model'].add(Embedding(vocab_size, EMBEDDING_DIM, input_length=t-1, weights=[embedding_matrix], trainable=False))
    result_dict[t]['model'].add(Dropout(0.2))
    result_dict[t]['model'].add(LSTM(64))
    result_dict[t]['model'].add(Dropout(0.2))
    result_dict[t]['model'].add(Dense(labels_len, activation='softmax'))
    result_dict[t]['model'].compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    result_dict[t]['model'].fit(padded_docs, labels, epochs=epoh_n, verbose=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Current type 0 All type 37
Epoch 1/50
828/828 [==============================] - 11s 11ms/step - loss: 8.8050 - accuracy: 0.0112
Epoch 2/50
828/828 [==============================] - 9s 11ms/step - loss: 8.3010 - accuracy: 0.0139
Epoch 3/50
828/828 [==============================] - 8s 10ms/step - loss: 8.2509 - accuracy: 0.0126
Epoch 4/50
828/828 [==============================] - 9s 11ms/step - loss: 8.2108 - accuracy: 0.0151
Epoch 5/50
828/828 [==============================] - 8s 10ms/step - loss: 8.1773 - accuracy: 0.0149
Epoch 6/50
828/828 [==============================] - 6s 7ms/step - loss: 8.1471 - accuracy: 0.0168
Epoch 7/50
828/828 [==============================] - 9s 11ms/step - loss: 8.1460 - accuracy: 0.0154
Epoch 8/50
828/828 [==============================] - 9s 11ms/step - loss: 8.1097 - accuracy: 0.0157
Epoch 9/50
828/828 [==============================] - 9s 11ms/step - loss: 8.0998 - accuracy: 0.0158
Epoch 10/50
828/828 [==============================] - 9s 11ms/s

344/344 [==============================] - 4s 11ms/step - loss: 7.5387 - accuracy: 0.0180
Epoch 12/50
344/344 [==============================] - 3s 9ms/step - loss: 7.5188 - accuracy: 0.0170
Epoch 13/50
344/344 [==============================] - 4s 11ms/step - loss: 7.4576 - accuracy: 0.0186
Epoch 14/50
344/344 [==============================] - 4s 11ms/step - loss: 7.4053 - accuracy: 0.0191
Epoch 15/50
344/344 [==============================] - 3s 9ms/step - loss: 7.3471 - accuracy: 0.0162
Epoch 16/50
344/344 [==============================] - 4s 11ms/step - loss: 7.2945 - accuracy: 0.0178
Epoch 17/50
344/344 [==============================] - 4s 10ms/step - loss: 7.2262 - accuracy: 0.0172
Epoch 18/50
344/344 [==============================] - 4s 10ms/step - loss: 7.2009 - accuracy: 0.0180
Epoch 19/50
344/344 [==============================] - 3s 9ms/step - loss: 7.1243 - accuracy: 0.0218
Epoch 20/50
344/344 [==============================] - 4s 11ms/step - loss: 7.0782 - accuracy: 0.

121/121 [==============================] - 1s 9ms/step - loss: 6.5832 - accuracy: 0.0201
Epoch 22/50
121/121 [==============================] - 1s 11ms/step - loss: 6.5542 - accuracy: 0.0176
Epoch 23/50
121/121 [==============================] - 1s 9ms/step - loss: 6.5076 - accuracy: 0.0192
Epoch 24/50
121/121 [==============================] - 1s 11ms/step - loss: 6.4739 - accuracy: 0.0209
Epoch 25/50
121/121 [==============================] - 1s 11ms/step - loss: 6.4371 - accuracy: 0.0193
Epoch 26/50
121/121 [==============================] - 1s 7ms/step - loss: 6.3617 - accuracy: 0.0237
Epoch 27/50
121/121 [==============================] - 1s 8ms/step - loss: 6.3423 - accuracy: 0.0245
Epoch 28/50
121/121 [==============================] - 1s 11ms/step - loss: 6.2356 - accuracy: 0.0248
Epoch 29/50
121/121 [==============================] - 1s 8ms/step - loss: 6.2337 - accuracy: 0.0261
Epoch 30/50
121/121 [==============================] - 1s 10ms/step - loss: 6.1837 - accuracy: 0.02

44/44 [==============================] - 0s 11ms/step - loss: 5.7493 - accuracy: 0.0339
Epoch 35/50
44/44 [==============================] - 1s 11ms/step - loss: 5.7000 - accuracy: 0.0345
Epoch 36/50
44/44 [==============================] - 0s 11ms/step - loss: 5.7489 - accuracy: 0.0218
Epoch 37/50
44/44 [==============================] - 0s 11ms/step - loss: 5.6166 - accuracy: 0.0330
Epoch 38/50
44/44 [==============================] - 0s 11ms/step - loss: 5.6738 - accuracy: 0.0294
Epoch 39/50
44/44 [==============================] - 0s 9ms/step - loss: 5.6201 - accuracy: 0.0456
Epoch 40/50
44/44 [==============================] - 0s 3ms/step - loss: 5.5697 - accuracy: 0.0328
Epoch 41/50
44/44 [==============================] - 0s 6ms/step - loss: 5.5551 - accuracy: 0.0355
Epoch 42/50
44/44 [==============================] - 0s 9ms/step - loss: 5.5059 - accuracy: 0.0387
Epoch 43/50
44/44 [==============================] - 0s 11ms/step - loss: 5.5474 - accuracy: 0.0422
Epoch 44/50
44/4

Epoch 16/50
22/22 [==============================] - 0s 12ms/step - loss: 6.0233 - accuracy: 0.0257
Epoch 17/50
22/22 [==============================] - 0s 12ms/step - loss: 5.9909 - accuracy: 0.0172
Epoch 18/50
22/22 [==============================] - 0s 12ms/step - loss: 5.9504 - accuracy: 0.0198
Epoch 19/50
22/22 [==============================] - 0s 11ms/step - loss: 5.9600 - accuracy: 0.0169
Epoch 20/50
22/22 [==============================] - 0s 12ms/step - loss: 5.9107 - accuracy: 0.0171
Epoch 21/50
22/22 [==============================] - 0s 12ms/step - loss: 5.8887 - accuracy: 0.0189
Epoch 22/50
22/22 [==============================] - 0s 11ms/step - loss: 5.8889 - accuracy: 0.0191
Epoch 23/50
22/22 [==============================] - 0s 11ms/step - loss: 5.8676 - accuracy: 0.0192
Epoch 24/50
22/22 [==============================] - 0s 11ms/step - loss: 5.8570 - accuracy: 0.0270
Epoch 25/50
22/22 [==============================] - 0s 11ms/step - loss: 5.7780 - accuracy: 0.0234


16/16 [==============================] - 0s 6ms/step - loss: 5.0502 - accuracy: 0.0300
Epoch 49/50
16/16 [==============================] - 0s 11ms/step - loss: 5.0663 - accuracy: 0.0453
Epoch 50/50
16/16 [==============================] - 0s 11ms/step - loss: 5.0693 - accuracy: 0.0385
Current type 13 All type 37
Epoch 1/50
12/12 [==============================] - 1s 3ms/step - loss: 5.7593 - accuracy: 0.0073 
Epoch 2/50
12/12 [==============================] - 0s 6ms/step - loss: 5.7284 - accuracy: 0.0044
Epoch 3/50
12/12 [==============================] - 0s 7ms/step - loss: 5.7123 - accuracy: 0.0084
Epoch 4/50
12/12 [==============================] - 0s 7ms/step - loss: 5.6985 - accuracy: 0.0209
Epoch 5/50
12/12 [==============================] - 0s 7ms/step - loss: 5.6704 - accuracy: 0.0270
Epoch 6/50
12/12 [==============================] - 0s 7ms/step - loss: 5.6697 - accuracy: 0.0222
Epoch 7/50
12/12 [==============================] - 0s 5ms/step - loss: 5.6575 - accuracy: 0.015

4/4 [==============================] - 0s 3ms/step - loss: 4.4925 - accuracy: 0.0000e+00
Epoch 4/10
4/4 [==============================] - 0s 4ms/step - loss: 4.5162 - accuracy: 0.0000e+00
Epoch 5/10
4/4 [==============================] - 0s 3ms/step - loss: 4.5237 - accuracy: 0.0247
Epoch 6/10
4/4 [==============================] - 0s 3ms/step - loss: 4.4901 - accuracy: 0.0154
Epoch 7/10
4/4 [==============================] - 0s 4ms/step - loss: 4.4792 - accuracy: 0.0309
Epoch 8/10
4/4 [==============================] - 0s 5ms/step - loss: 4.5303 - accuracy: 0.0246
Epoch 9/10
4/4 [==============================] - 0s 5ms/step - loss: 4.5192 - accuracy: 0.0255
Epoch 10/10
4/4 [==============================] - 0s 5ms/step - loss: 4.4702 - accuracy: 0.0246
Current type 18 All type 37
Epoch 1/10
3/3 [==============================] - 1s 4ms/step - loss: 4.2991 - accuracy: 0.0583
Epoch 2/10
3/3 [==============================] - 0s 3ms/step - loss: 4.3409 - accuracy: 0.0107
Epoch 3/10
3/3

1/1 [==============================] - 0s 4ms/step - loss: 0.7333 - accuracy: 0.5000
Epoch 8/10
1/1 [==============================] - 0s 4ms/step - loss: 0.5582 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 3ms/step - loss: 0.6193 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 4ms/step - loss: 0.7463 - accuracy: 0.5000
Current type 34 All type 37
Epoch 1/10
1/1 [==============================] - 1s 916ms/step - loss: 0.8239 - accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 0s 4ms/step - loss: 0.6738 - accuracy: 0.5000
Epoch 3/10
1/1 [==============================] - 0s 4ms/step - loss: 0.8144 - accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 4ms/step - loss: 0.7349 - accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 4ms/step - loss: 0.5593 - accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 4ms/step - loss: 0.8219 - accuracy: 0.0000e+00
Epoch 7/

In [11]:
f = open('output.tsv', 'w')

n_tokens = 0
hits = 0

for line in tqdm(open('../data/dev.tsv').readlines()):
    # Split into two columns
    row = line.strip().split('\t')
    # Our tokens are in column one, split by space
    tokens = row[0].split(' ')
    # The test tokens are the beginning of sentence symbol + the list of tokens
    tst_tokens = ['#'] + tokens
    # Increment the number of tokens by the length of the list containing the tokens
    n_tokens += len(tokens)

    # This is our output
    output = []

    # For each of the tokens in the "tst_tokens" list (e.g. the list + the beginning of sentence symbol)
    for i in range(len(tst_tokens)-1):
        first = tst_tokens[i]  # First token in bigram
        second = tst_tokens[i+1]  # Second token in bigram

        if i == 0:
            predicted_second = pd.Series(result_dict_common[0]).value_counts()[:1].index[0]
        else:
            exits_text = ' '.join(tst_tokens[1:i+1])
            predicted_second = result_dict[i+1]['model'].predict(pad_sequences(tokenizer.texts_to_sequences([exits_text]), maxlen=i, padding='post'))
            predicted_second = np.argmax(predicted_second)
            predicted_second = result_dict[i+1]['le'].inverse_transform([predicted_second])
            predicted_second = predicted_second[0]
        

        if predicted_second == second:
            # We add this whole token to the output
            # e.g. a single click on a prediction
            output.append(predicted_second)
            # Increment the number of hits by 1
            hits += 1
        else:
            # Otherwise we add each individual character to the output
            # e.g. writing out each of the individual clicks
            output += [c for c in second]

        output.append('_')

    
    print('%s\t%s' % (row[0], ' '.join(output)), file=f)

f.close()

  0%|          | 0/1000 [00:00<?, ?it/s]

  3%|▎         | 26/1000 [00:04<01:18, 12.42it/s]

100%|██████████| 1000/1000 [01:59<00:00,  8.36it/s]


In [12]:
hits

33

In [13]:
!python3 ../evaluate.py ../data/dev.tsv ./output.tsv

Characters: 37897
Tokens: 8788
Clicks: 37759
Clicks/Token: 4.2966545289030496
Clicks/Character: 0.9963585508087711


In [14]:
f = open('output_test.tsv', 'w')

n_tokens = 0
hits = 0

for line in tqdm(open('../data/test/test.tsv').readlines()):
    # Split into two columns
    row = line.strip().split('\t')
    # Our tokens are in column one, split by space
    tokens = row[0].split(' ')
    # The test tokens are the beginning of sentence symbol + the list of tokens
    tst_tokens = ['#'] + tokens
    # Increment the number of tokens by the length of the list containing the tokens
    n_tokens += len(tokens)

    # This is our output
    output = []

    # For each of the tokens in the "tst_tokens" list (e.g. the list + the beginning of sentence symbol)
    for i in range(len(tst_tokens)-1):
        first = tst_tokens[i]  # First token in bigram
        second = tst_tokens[i+1]  # Second token in bigram

        if i == 0:
            predicted_second = pd.Series(result_dict_common[0]).value_counts()[:1].index[0]
        else:
            exits_text = ' '.join(tst_tokens[1:i+1])
            predicted_second = result_dict[i+1]['model'].predict(pad_sequences(tokenizer.texts_to_sequences([exits_text]), maxlen=i, padding='post'))
            predicted_second = np.argmax(predicted_second)
            predicted_second = result_dict[i+1]['le'].inverse_transform([predicted_second])
            predicted_second = predicted_second[0]
        

        if predicted_second == second:
            # We add this whole token to the output
            # e.g. a single click on a prediction
            output.append(predicted_second)
            # Increment the number of hits by 1
            hits += 1
        else:
            # Otherwise we add each individual character to the output
            # e.g. writing out each of the individual clicks
            output += [c for c in second]

        output.append('_')

    
    print('%s\t%s' % (row[0], ' '.join(output)), file=f)

f.close()

100%|██████████| 1006/1006 [01:46<00:00,  9.49it/s]


In [15]:
hits

22

In [16]:
!python3 ../evaluate.py ../data/test/test.tsv ./output_test.tsv

Characters: 37927
Tokens: 8374
Clicks: 37834
Clicks/Token: 4.518032003821352
Clicks/Character: 0.9975479210061434
